# Setup and load data:

In [ ]:
%load_ext autoreload
%autoreload 2

from functions.functions import *
from functions.plotting import *
from functions.autoencoder import *
from functions.mdn import *
from keras import backend as K
from keras.callbacks import TerminateOnNaN, ModelCheckpoint
from keras.models import model_from_json
from livelossplot.keras import PlotLossesCallback
import livelossplot
import tensorflow as tf
import numpy as np
import os
import glob
from itertools import combinations
import matplotlib
plt.rcParams['animation.ffmpeg_path'] = '/gpfs/loomis/project/hep/demers/mnp3/conda_envs/choreo/bin/ffmpeg' # for using html5 video in Jupyter notebook
print(matplotlib.animation.writers.list()) # check that ffmpeg is loaded. if it's not there, use .to_jshtml() instead of .to_html5_video().

In [ ]:
setup_gpus()

In [ ]:
data = load_data('data/mariel_*')

# Check out the real data:

In [ ]:
frame = np.random.randint(0,data.selected.X.shape[1]-50)
print("Starting from frame {}...".format(frame))

# HTML(animate(data.full.X[:,frame:,:], frames=100))
HTML(animate(data.selected.X[:,frame:,:], frames=100, edges=data.selected.edges, colors='black'))

### Define the training dataset:

In [ ]:
# prepare data for the LSTM_MDN
X = data.selected.X
X.shape
X = X.swapaxes(0, 1) # reshape to time, vert, dim
n_time, n_verts, n_dims = X.shape
look_back = 128
n_mixes = 6

# center each frame along the x and y axes to simplify training
X[:,:,0] = X[:,:,0] - np.mean(X[:,:,0], axis=0) + 0.5*np.ones(n_verts)
X[:,:,1] = X[:,:,1] - np.mean(X[:,:,1], axis=0) + 0.5*np.ones(n_verts)

# lstm expects data in shape [samples_in_batch, timestamps, values]
train_X = []
train_Y = []
for i in range(look_back, n_time, 1):
    train_X.append( X[i-look_back:i,:,:].reshape(look_back, n_verts * n_dims) ) # look_back, verts * dims
    train_Y.append( X[i,:,:].reshape(n_verts * n_dims) ) # verts * dims
train_X = np.array(train_X) # n_samples, lookback, verts * dims
train_Y = np.array(train_Y) # n_samples, verts * dims

print(train_X.shape)
print(train_Y.shape)

# Load a trained model + weights

In [ ]:
trained_model = model_from_json(open('models/model-32chor-rnn.json').read(), {'MDN': MDN, 'LSTM_MDN': LSTM_MDN})
trained_model.load_weights('weights/weights-32chor-rnn.h5')
trained_model.summary()

### See how well the model can predict the next frame in the input sequence:

In [ ]:
# visualize how well the model learned the input sequence
n_frames = 100 # n frames of time slices to generate
output_dims = train_X.shape[2]
frame = np.random.randint(0,data.full.X.shape[1]-50)
frames = []

test_X = train_X[frame:frame+n_frames] # data to pass into forward prop through the model
y_pred = trained_model.predict(test_X) # output with shape (n_frames, (output_dims+2) * n_mixes )

# partition out the mus, sigs, and mixture weights
for i in range(n_frames):
    y = y_pred[i].squeeze()
    mus = y[:n_mixes*output_dims]
    sigs = y[n_mixes*output_dims:n_mixes*output_dims + n_mixes]
    alphas = y[-n_mixes:]
    # find the most likely distribution
    alpha_idx = np.argmax(alphas)
    # pull out the mus that correspond to the selected alpha index
    positions = mus[alpha_idx * output_dims:(alpha_idx+1) * output_dims]
    frames.append(positions)
#     print(alphas)
frames = np.array(frames)
lstm_predictions = np.dstack((frames.T[::3,:],frames.T[1::3,:],frames.T[2::3,:]))
HTML(animate_ghost(data.selected.X[:,frame:,:], lstm_predictions, frames=n_frames, edges=data.selected.edges, colors='blue', ghost_shift = 0.3))

# Now generate new sequences!

In [ ]:
n_frames = 100 # n frames of time slices to generate
frames = []
seed = np.random.randint(0, len(train_X)-1)
x = np.expand_dims(train_X[seed], axis=0)
print(' * seeding with', seed)

for i in range(n_frames):
    y = trained_model.predict(x).squeeze()
    mus = y[:n_mixes*output_dims]
    sigs = y[n_mixes*output_dims:-n_mixes]
    alphas = softmax(y[-n_mixes:])

    # select the alpha channel to use
    alpha_idx = np.argmax(alphas)
#     print(alphas)
    # grab the mus and sigs associated with the selected alpha_idx
    frame_mus = mus.ravel()[alpha_idx*output_dims : (alpha_idx+1)*output_dims]
    frame_sig = sigs[alpha_idx] / 100

    # now sample from each Gaussian
    positions = [np.random.normal(loc=m, scale=frame_sig) for m in frame_mus]
    positions = frame_mus

    # add these positions to the results
    frames.append(positions)

    # pull out a new training example - stack the new result on
    # all values after the first from the bottom-most value in the x's
    start = x[:,1:,:]
    end = np.expand_dims( np.expand_dims(positions, axis=0), axis=0 )
    x = np.concatenate((start, end), axis=1)
    
frames = np.array(frames)
lstm_predictions = np.dstack((frames.T[::3,:],frames.T[1::3,:],frames.T[2::3,:]))
prompt_plus_generated_seq = np.concatenate((data.selected.X[:,seed:seed+look_back,:],lstm_predictions), axis=1)
HTML(animate_ghost(data.selected.X[:,seed:seed+look_back+n_frames:,:], prompt_plus_generated_seq, frames=look_back+n_frames, edges=data.selected.edges, colors='blue'))

In [ ]:
HTML(animate(lstm_predictions, frames=n_frames, edges=data.selected.edges, colors='blue'))